## Парсинг данных с сайта на Python. Модуль А.

### Задание

Необходимо собрать данные о вакансиях BI-аналитиков или о вакансиях 
экскурсоводов из открытых источников.
В базе необходимо:
- провести предобработку данных (привести все к нижнему регистру, 
задать одинаковый формат данных),
- необходимо определить, какие пропущенные значения являются 
критическими и провести с ними работу, оценить погрешность при 
допустимом количестве пропущенных значений
- преобразовать типы данных (при необходимости).

Определены и обоснованы наиболее значимые атрибуты.

Для каждого атрибута присутствует текстовое описание, информация о количестве пустых значений, анализ плотности распределения значений.

При сборе данных использованы запросы к данным.

Сбор данных выполнен программно.

Предобработка данных выполнена программно.

### Этапы парсинга

1. Получение страниц в виде json файлов
2. Получение вакансий с каждой страницы в виде json файлов
3. Формирование датафрейма по информации каждой вакансии
4. Обработка текста и нулевых значений

In [2]:
# Библиотека для работы с HTTP-запросами. Будем использовать ее для обращения к API HH
import requests

# Пакет для удобной работы с данными в формате json
import json

# Модуль для работы со значением времени
import time

# Модуль для работы с операционной системой. Будем использовать для работы с файлами
import os

# Модуль для работы с датафреймами
import pandas as pd

# Модуль для определения эмоционального окраса описания
from textblob import TextBlob

# Модули для отчистки и вывода информации
from IPython.display import clear_output
from IPython.display import display

### Получение информаций со страниц

Получим страницы в виде json файлов через API hh.ru. Можете ознакомиться с документацией API HH на их сайте.

In [10]:
# Создаём папку в корне для удобной работы с файлами
if not os.path.exists('./docs'):
    os.makedirs('./docs')
# Создаём папку в docs для удобной работы с файлами (страницы)
if not os.path.exists('./docs/pagination'):
    os.makedirs('./docs/pagination')
 
def getPage(page = 0):
    """
    Создаем метод для получения страницы со списком вакансий.
    Аргументы:
        page - Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница
    """
    
    # Справочник для параметров GET-запроса
    params = {
        'text': 'NAME:Аналитик', # Текст фильтра. В имени должно быть слово "Аналитик"
        'area': 1, # Поиск ощуществляется по вакансиям города Москва
        'page': page, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
    }
    
    
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    req.close()
    return data


# Считываем первые 2000 вакансий
for page in range(0, 20):
    
    # Преобразуем текст ответа запроса в справочник Python
    jsObj = json.loads(getPage(page))
    
    # Сохраняем файлы в папку {путь до текущего документа со скриптом}\docs\pagination
    # Определяем количество файлов в папке для сохранения документа с ответом запроса
    # Полученное значение используем для формирования имени документа
    nextFileName = './docs/pagination/{}.json'.format(len(os.listdir('./docs/pagination')))
    
    # Создаем новый документ, записываем в него ответ запроса, после закрываем
    f = open(nextFileName, mode='w', encoding='utf8')
    f.write(json.dumps(jsObj, ensure_ascii=False))
    f.close()
    
    # Проверка на последнюю страницу, если вакансий меньше 2000
    if (jsObj['pages'] - page) <= 1:
        break
    
    # Необязательная задержка, но чтобы не нагружать сервисы hh, оставим. 5 сек мы может подождать
    time.sleep(0.25)
    
print('Старницы поиска собраны')

Старницы поиска собраны


### Получение вакансий из файлов страниц

In [12]:
# Создаём папку в docs для удобной работы с файлами (вакансии)
if not os.path.exists('./docs/vacancies'):
    os.makedirs('./docs/vacancies')
    
# Получаем перечень ранее созданных файлов со списком вакансий и проходимся по нему в цикле 
for fl in os.listdir('./docs/pagination'):
    
    # Открываем файл, читаем его содержимое, закрываем файл
    f = open('./docs/pagination/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
    
    # Преобразуем полученный текст в объект справочника
    jsonObj = json.loads(jsonText)
    
    # Получаем и проходимся по непосредственно списку вакансий
    for v in jsonObj['items']:
        
        # Обращаемся к API и получаем детальную информацию по конкретной вакансии
        req = requests.get(v['url'])
        data = req.content.decode()
        req.close()
        
        # Создаем файл в формате json с идентификатором вакансии в качестве названия
        # Записываем в него ответ запроса и закрываем файл
        fileName = './docs/vacancies/{}.json'.format(v['id'])
        f = open(fileName, mode='w', encoding='utf8')
        f.write(data)
        f.close()
        
        time.sleep(0.25)
        
print('Вакансии собраны')

Вакансии собраны


### Удаление файлов, не прошедших проверку (где требуется captcha)

Обычно такие файлы весят меньше 2 КБ, поэтому будем удалять именно их. (Можно убедиться что файлы у которых вес меньше 2 КБ нет нужных нам параметров таких как описание, название и т.д., вы можете открыть файл в блокноте и убедиться в этом сами)

In [4]:
def delete_files_smaller_than_2kb(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and os.path.getsize(file_path) < 2 * 1024:
            os.remove(file_path)

# Пример использования
delete_files_smaller_than_2kb('./docs/vacancies')

In [5]:
cnt_docs = len(os.listdir('./docs/vacancies'))
print(cnt_docs)

1202


### Формирование датафрейма

Теперь из наших файлов можно сформировать датасет с нужной информацией по вакансиям

In [3]:
IDs = [] # Список идентификаторов вакансий
names = [] # Список наименований вакансий
descriptions = [] # Список описаний вакансий
experience = [] # Список требуемого опыта вакансий
company_name = [] # Список компаний вакансий

skills_vac = [] # Список идентификаторов вакансий
skills_name = [] # Список названий навыков
schedule_day = [] # Список графика работы вакансий
address_city = [] # Список городов вакансий
salary = [] # Список зарплат (от) вакансий
currency = [] # Список типов валют вакансий
address_street = [] # Список улиц вакансий
date_public = [] # Список дат публикации вакансий

# В выводе будем отображать прогресс
# Для этого узнаем общее количество файлов, которые надо обработать
# Счетчик обработанных файлов установим в ноль
cnt_docs = len(os.listdir('./docs/vacancies'))
i = 0

# Проходимся по всем файлам в папке vacancies
for fl in os.listdir('./docs/vacancies'):
    
    # Открываем, читаем и закрываем файл
    f = open('./docs/vacancies/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
    
    # Текст файла переводим в справочник
    jsonObj = json.loads(jsonText)
    
    # Заполняем списки
    IDs.append(jsonObj['id'])
    names.append(jsonObj['name'])
    descriptions.append(jsonObj['description'])
    schedule_day.append(jsonObj['schedule']['name'])
    experience.append(jsonObj['experience']['name'])
    
    if(jsonObj['department'] == None):
        company_name.append(None)
    else:
        company_name.append(jsonObj['department']['name'])
    
    if (jsonObj['salary'] == None):
        salary.append(None)
        currency.append(None)
    else:
        salary.append(jsonObj['salary']['from'])
        currency.append(jsonObj['salary']['currency'])

    
    if(jsonObj['address'] == None):
        address_city.append(None)
        address_street.append(None)
    else:
        address_city.append(jsonObj['address']['city'])
        address_street.append(jsonObj['address']['street'])
    
    date_public.append(jsonObj['published_at'])
    
    # Т.к. навыки хранятся в виде массива, то проходимся по нему циклом
    skills_oner = []
    if len(jsonObj['key_skills']) == 0:
        skills_name.append("Не указано")
    else: 
        for skl in jsonObj['key_skills']:
            skills_oner.append(skl['name'])
            
        skills_string = ', '.join(skills_oner)
        skills_name.append(skills_string)
    
    # Увеличиваем счетчик обработанных файлов на 1, очищаем вывод ячейки и выводим прогресс
    i += 1
    clear_output(wait=True)
    display('Готово {} из {}'.format(i, cnt_docs))

'Готово 1202 из 1202'

Сформируем датафрейм из наших получившихся списков.

In [4]:
df = pd.DataFrame({'id': IDs, 'company': company_name, 'name': names, 'description': descriptions, 'skills': skills_name, 'date_public': date_public, 'schedule': schedule_day, 'experience': experience, 'city': address_city, 'street': address_street, 'salary': salary, 'currency': currency})

In [71]:
df.head(10)

,id,company,name,description,skills,date_public,schedule,experience,city,street,salary,currency
0,73013752,None,Аналитик в планово-диспетчерский отдел,"<strong>Обязанности:</strong> <ul> <li>Расчет,...","Пользователь ПК, Электронный документооборот, ...",2024-04-05T08:55:42+0300,Полный день,От 1 года до 3 лет,Москва,Сущёвская улица,116000.0,RUR
1,76533535,None,Аналитик в команду внедрения ботов,<p><strong>Banks Soft Systems</strong> - это в...,"Написание сценариев, Разработка технических за...",2024-03-25T11:07:32+0300,Удаленная работа,От 1 года до 3 лет,None,None,NaN,None
2,77195081,None,Системный аналитик в IT Компанию РОЛЬФТЕХ (уда...,<p><em><strong>Наша команда расширяется и мы н...,Не указано,2024-04-17T09:34:24+0300,Полный день,От 1 года до 3 лет,Москва,Алтуфьевское шоссе,NaN,None
3,77538397,"Компания «СПОРТМАСТЕР», Офис",Бизнес-аналитик,<p>«Спортмастер» — это международный спортивны...,"MS Visio, Business Studio, MS PowerPoint, Опти...",2024-03-27T09:32:37+0300,Полный день,От 1 года до 3 лет,None,None,NaN,None
4,81311010,ДИКСИ. IT,Разработчик-аналитик SQL (Junior),<strong>Функции:</strong> <ul> <li>Разработка ...,"Python, SQL, SSIS, MS SQL Server, C#, SSAS, SS...",2024-03-22T12:42:39+0300,Удаленная работа,От 3 до 6 лет,None,None,NaN,None
5,82124306,None,Продуктовый аналитик (PREMIER онлайн-кинотеатр),<p><strong>PREMIER</strong> – <em>это видеосер...,"SQL, Tableau, PowerBI, Power BI, ClickHouse, D...",2024-04-15T13:17:52+0300,Полный день,От 1 года до 3 лет,Москва,Ленинградский проспект,NaN,None
6,82175755,None,Аналитик данных (SQL),<p>Ищем Аналитика в Департамент риск-отчётност...,"SQL, аналитика данных, Анализ рисков",2024-04-01T09:37:28+0300,Полный день,От 1 года до 3 лет,None,None,NaN,None
7,82231533,None,Аналитик/Разработчик BI,<p>GlowByte - лидер в сфере разработки Busines...,"SQL, Business intelligence, BI",2024-04-16T12:26:48+0300,Полный день,От 1 года до 3 лет,None,None,NaN,None
8,82742802,None,Лингвист-аналитик,<p>Российская компания <strong>Megaputer Intel...,Не указано,2024-04-18T11:48:10+0300,Полный день,От 1 года до 3 лет,None,None,95000.0,RUR
9,84170028,ГК Иннотех | Мультипродакт,Главный системный аналитик (импортозамещение),<p>В направление импортозамещения ищем Системн...,Linux,2024-04-10T18:35:51+0300,Полный день,От 3 до 6 лет,None,None,NaN,None


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           1202 non-null   object 
 1   company      320 non-null    object 
 2   name         1202 non-null   object 
 3   description  1202 non-null   object 
 4   skills       1202 non-null   object 
 5   date_public  1202 non-null   object 
 6   schedule     1202 non-null   object 
 7   experience   1202 non-null   object 
 8   city         521 non-null    object 
 9   street       515 non-null    object 
 10  salary       296 non-null    float64
 11  currency     365 non-null    object 
dtypes: float64(1), object(11)
memory usage: 112.8+ KB


Присутствуют пустые значения в атрибутах company, city, street, salary, currency. Скорее всего это из-за того что работодатели не указывали в заявлениях эти даннные, поэтому просто будем заполнять и как "Не указано", но в salary будет заполнять как 0, для того чтобы тип столбца остался числовым для удобной работы в дашборде.

In [5]:
# Замена значений None и NaN на "Не указано", исключая столбец salary
df.loc[:, ['company', 'name', 'skills', 'date_public', 'schedule', 'experience', 'city', 'street', 'currency']] = df.loc[:, ['company', 'name', 'skills', 'date_public', 'schedule', 'experience', 'city', 'street', 'currency']].fillna(value='Не указано')

# Замена значений NaN на 0 в столбце salary
df.loc[:, ['salary']] = df.loc[:, ['salary']].fillna(value=0)

In [74]:
# Подсчет количества пустых значений в каждом столбце
print(df.isnull().sum())

id             0
company        0
name           0
description    0
skills         0
date_public    0
schedule       0
experience     0
city           0
street         0
salary         0
currency       0
dtype: int64


Как можно заметить, мы избавились от пустых значений в датафрейме.

In [75]:
df.head(5)

,id,company,name,description,skills,date_public,schedule,experience,city,street,salary,currency
0,73013752,Не указано,Аналитик в планово-диспетчерский отдел,"<strong>Обязанности:</strong> <ul> <li>Расчет,...","Пользователь ПК, Электронный документооборот, ...",2024-04-05T08:55:42+0300,Полный день,От 1 года до 3 лет,Москва,Сущёвская улица,116000.0,RUR
1,76533535,Не указано,Аналитик в команду внедрения ботов,<p><strong>Banks Soft Systems</strong> - это в...,"Написание сценариев, Разработка технических за...",2024-03-25T11:07:32+0300,Удаленная работа,От 1 года до 3 лет,Не указано,Не указано,0.0,Не указано
2,77195081,Не указано,Системный аналитик в IT Компанию РОЛЬФТЕХ (уда...,<p><em><strong>Наша команда расширяется и мы н...,Не указано,2024-04-17T09:34:24+0300,Полный день,От 1 года до 3 лет,Москва,Алтуфьевское шоссе,0.0,Не указано
3,77538397,"Компания «СПОРТМАСТЕР», Офис",Бизнес-аналитик,<p>«Спортмастер» — это международный спортивны...,"MS Visio, Business Studio, MS PowerPoint, Опти...",2024-03-27T09:32:37+0300,Полный день,От 1 года до 3 лет,Не указано,Не указано,0.0,Не указано
4,81311010,ДИКСИ. IT,Разработчик-аналитик SQL (Junior),<strong>Функции:</strong> <ul> <li>Разработка ...,"Python, SQL, SSIS, MS SQL Server, C#, SSAS, SS...",2024-03-22T12:42:39+0300,Удаленная работа,От 3 до 6 лет,Не указано,Не указано,0.0,Не указано


Теперь нужно удалить HTML теги из описания, сделаем это через библиотеку BeautifulSoup4.

In [6]:
from bs4 import BeautifulSoup

In [7]:
# Определяем функцию для удаления тегов HTML
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

# Применяем функцию к столбцу 'description'
df['description'] = df['description'].apply(remove_html_tags)

In [78]:
df.head(10)

,id,company,name,description,skills,date_public,schedule,experience,city,street,salary,currency
0,73013752,Не указано,Аналитик в планово-диспетчерский отдел,"Обязанности: Расчет, формирование , контроль ...","Пользователь ПК, Электронный документооборот, ...",2024-04-05T08:55:42+0300,Полный день,От 1 года до 3 лет,Москва,Сущёвская улица,116000.0,RUR
1,76533535,Не указано,Аналитик в команду внедрения ботов,Banks Soft Systems - это ведущий разработчик в...,"Написание сценариев, Разработка технических за...",2024-03-25T11:07:32+0300,Удаленная работа,От 1 года до 3 лет,Не указано,Не указано,0.0,Не указано
2,77195081,Не указано,Системный аналитик в IT Компанию РОЛЬФТЕХ (уда...,Наша команда расширяется и мы находимся в поис...,Не указано,2024-04-17T09:34:24+0300,Полный день,От 1 года до 3 лет,Москва,Алтуфьевское шоссе,0.0,Не указано
3,77538397,"Компания «СПОРТМАСТЕР», Офис",Бизнес-аналитик,«Спортмастер» — это международный спортивный р...,"MS Visio, Business Studio, MS PowerPoint, Опти...",2024-03-27T09:32:37+0300,Полный день,От 1 года до 3 лет,Не указано,Не указано,0.0,Не указано
4,81311010,ДИКСИ. IT,Разработчик-аналитик SQL (Junior),"Функции: Разработка SSRS, SSAS отчетности Соз...","Python, SQL, SSIS, MS SQL Server, C#, SSAS, SS...",2024-03-22T12:42:39+0300,Удаленная работа,От 3 до 6 лет,Не указано,Не указано,0.0,Не указано
5,82124306,Не указано,Продуктовый аналитик (PREMIER онлайн-кинотеатр),PREMIER – это видеосервис с новыми русскими се...,"SQL, Tableau, PowerBI, Power BI, ClickHouse, D...",2024-04-15T13:17:52+0300,Полный день,От 1 года до 3 лет,Москва,Ленинградский проспект,0.0,Не указано
6,82175755,Не указано,Аналитик данных (SQL),Ищем Аналитика в Департамент риск-отчётности Ч...,"SQL, аналитика данных, Анализ рисков",2024-04-01T09:37:28+0300,Полный день,От 1 года до 3 лет,Не указано,Не указано,0.0,Не указано
7,82231533,Не указано,Аналитик/Разработчик BI,GlowByte - лидер в сфере разработки Business I...,"SQL, Business intelligence, BI",2024-04-16T12:26:48+0300,Полный день,От 1 года до 3 лет,Не указано,Не указано,0.0,Не указано
8,82742802,Не указано,Лингвист-аналитик,"Российская компания Megaputer Intelligence, од...",Не указано,2024-04-18T11:48:10+0300,Полный день,От 1 года до 3 лет,Не указано,Не указано,95000.0,RUR
9,84170028,ГК Иннотех | Мультипродакт,Главный системный аналитик (импортозамещение),В направление импортозамещения ищем Системного...,Linux,2024-04-10T18:35:51+0300,Полный день,От 3 до 6 лет,Не указано,Не указано,0.0,Не указано


Теперь все html теги убраны из описания (description)

Удалим строки у которых валюта не в рублях для более удобной работы с данными (их довольно мало, поэтому можно просто удалить). Где валюта "Не указано" будем думать что там рубли, но для достоверности лучше оставить "Не указано" (если вдруг пользователь захочет быть на 100% уверенным что ЗП будет именно в той валюте, которая указана)

In [8]:
df = df[df['currency'].isin(['RUR', 'Не указано'])]

In [80]:
len(df)

1193

In [60]:
df.head()

,id,company,name,description,skills,date_public,schedule,experience,city,street,salary,currency
0,73013752,Не указано,Аналитик в планово-диспетчерский отдел,"Обязанности: Расчет, формирование , контроль ...","Пользователь ПК, Электронный документооборот, ...",2024-04-05T08:55:42+0300,Полный день,От 1 года до 3 лет,Москва,Сущёвская улица,116000.0,RUR
1,76533535,Не указано,Аналитик в команду внедрения ботов,Banks Soft Systems - это ведущий разработчик в...,"Написание сценариев, Разработка технических за...",2024-03-25T11:07:32+0300,Удаленная работа,От 1 года до 3 лет,Не указано,Не указано,0.0,Не указано
2,77195081,Не указано,Системный аналитик в IT Компанию РОЛЬФТЕХ (уда...,Наша команда расширяется и мы находимся в поис...,Не указано,2024-04-17T09:34:24+0300,Полный день,От 1 года до 3 лет,Москва,Алтуфьевское шоссе,0.0,Не указано
3,77538397,"Компания «СПОРТМАСТЕР», Офис",Бизнес-аналитик,«Спортмастер» — это международный спортивный р...,"MS Visio, Business Studio, MS PowerPoint, Опти...",2024-03-27T09:32:37+0300,Полный день,От 1 года до 3 лет,Не указано,Не указано,0.0,Не указано
4,81311010,ДИКСИ. IT,Разработчик-аналитик SQL (Junior),"Функции: Разработка SSRS, SSAS отчетности Соз...","Python, SQL, SSIS, MS SQL Server, C#, SSAS, SS...",2024-03-22T12:42:39+0300,Удаленная работа,От 3 до 6 лет,Не указано,Не указано,0.0,Не указано


Теперь преобразуем нашу дату в нормальный вид для даты, без времени и часового пояса.

In [9]:
# Изменяем тип данных столбца 'date_public' на datetime64[ns]
df['date_public'] = pd.to_datetime(df['date_public'], format='%Y-%m-%dT%H:%M:%S%z')

# Преобразуем столбец 'date_public' в формат даты
df['date_public'] = df['date_public'].dt.date

In [82]:
df.head(5)

,id,company,name,description,skills,date_public,schedule,experience,city,street,salary,currency
0,73013752,Не указано,Аналитик в планово-диспетчерский отдел,"Обязанности: Расчет, формирование , контроль ...","Пользователь ПК, Электронный документооборот, ...",2024-04-05,Полный день,От 1 года до 3 лет,Москва,Сущёвская улица,116000.0,RUR
1,76533535,Не указано,Аналитик в команду внедрения ботов,Banks Soft Systems - это ведущий разработчик в...,"Написание сценариев, Разработка технических за...",2024-03-25,Удаленная работа,От 1 года до 3 лет,Не указано,Не указано,0.0,Не указано
2,77195081,Не указано,Системный аналитик в IT Компанию РОЛЬФТЕХ (уда...,Наша команда расширяется и мы находимся в поис...,Не указано,2024-04-17,Полный день,От 1 года до 3 лет,Москва,Алтуфьевское шоссе,0.0,Не указано
3,77538397,"Компания «СПОРТМАСТЕР», Офис",Бизнес-аналитик,«Спортмастер» — это международный спортивный р...,"MS Visio, Business Studio, MS PowerPoint, Опти...",2024-03-27,Полный день,От 1 года до 3 лет,Не указано,Не указано,0.0,Не указано
4,81311010,ДИКСИ. IT,Разработчик-аналитик SQL (Junior),"Функции: Разработка SSRS, SSAS отчетности Соз...","Python, SQL, SSIS, MS SQL Server, C#, SSAS, SS...",2024-03-22,Удаленная работа,От 3 до 6 лет,Не указано,Не указано,0.0,Не указано


### Эмоциональный окрас текста описания

In [11]:
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        return "positive"
    elif sentiment < 0:
        return "negative"
    else:
        return "neutral"

# Создаем столбец sentiment_description
df['sentiment_description'] = df['description'].apply(get_sentiment)

In [12]:
df.head(5)

,id,company,name,description,skills,date_public,schedule,experience,city,street,salary,currency,sentiment_description
0,73013752,Не указано,Аналитик в планово-диспетчерский отдел,"Обязанности: Расчет, формирование , контроль ...","Пользователь ПК, Электронный документооборот, ...",2024-04-05,Полный день,От 1 года до 3 лет,Москва,Сущёвская улица,116000.0,RUR,neutral
1,76533535,Не указано,Аналитик в команду внедрения ботов,Banks Soft Systems - это ведущий разработчик в...,"Написание сценариев, Разработка технических за...",2024-03-25,Удаленная работа,От 1 года до 3 лет,Не указано,Не указано,0.0,Не указано,positive
2,77195081,Не указано,Системный аналитик в IT Компанию РОЛЬФТЕХ (уда...,Наша команда расширяется и мы находимся в поис...,Не указано,2024-04-17,Полный день,От 1 года до 3 лет,Москва,Алтуфьевское шоссе,0.0,Не указано,neutral
3,77538397,"Компания «СПОРТМАСТЕР», Офис",Бизнес-аналитик,«Спортмастер» — это международный спортивный р...,"MS Visio, Business Studio, MS PowerPoint, Опти...",2024-03-27,Полный день,От 1 года до 3 лет,Не указано,Не указано,0.0,Не указано,neutral
4,81311010,ДИКСИ. IT,Разработчик-аналитик SQL (Junior),"Функции: Разработка SSRS, SSAS отчетности Соз...","Python, SQL, SSIS, MS SQL Server, C#, SSAS, SS...",2024-03-22,Удаленная работа,От 3 до 6 лет,Не указано,Не указано,0.0,Не указано,neutral


### Значимые атрибуты

В нашем датафрейме при парсинге я старался брать значимые атрибуты (для дальнейшего вывода и работы с данными в дашборде). Поэтому в моём датафрейме все атрибуты являются значимыми. Бдует фильтрация по дате публикации, по опыту работы, по ЗП и т.д. Айди оставил для удобной работы с фильтрацией в дашборде (через запросы), хотя его можно было и не добавлять, но пусть будет, вдруг в дальнейшем будет работа с БД.

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1193 entries, 0 to 1201
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           1193 non-null   object 
 1   company      1193 non-null   object 
 2   name         1193 non-null   object 
 3   description  1193 non-null   object 
 4   skills       1193 non-null   object 
 5   date_public  1193 non-null   object 
 6   schedule     1193 non-null   object 
 7   experience   1193 non-null   object 
 8   city         1193 non-null   object 
 9   street       1193 non-null   object 
 10  salary       1193 non-null   float64
 11  currency     1193 non-null   object 
dtypes: float64(1), object(11)
memory usage: 121.2+ KB


### Сохранение датафрейма в виде excel файла

Датафрейм готов, теперь можно сохранить его в виде csv файла и уже дальше работать с этими данными в дашборде.

In [84]:
# Сохраняем датафрейм в файл Excel с учетом кодировки
df.to_excel('bi_analytics.xlsx', encoding='utf-8', index=False)

C:\Users\1\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
